In [1]:
import os
import sys
print(os.getcwd())
sys.path.append("../wanpy")

import numpy as np
from pythtb import *
from wpythtb import Bloch, K_mesh, Wannier
import models

import matplotlib.pyplot as plt
from itertools import product

/Users/treycole/Repos/WanPy/dev


In [9]:
# tight-binding parameters
delta = 1
t = 1
t2 = -0.3

n_super_cell = 2
model = models.Haldane(delta, t, t2).make_supercell([[n_super_cell, 0], [0, n_super_cell]])
model.onsite = model.set_onsite(0.5, ind_i = 2, mode='add')

#############

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)
lat_vecs = model.get_lat()
recip_lat_vecs = model.get_recip_lat_vecs()
orb_vecs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)

nkx = nky = 10
bloch_wfs = Bloch(model, nkx, nky)
bloch_wfs.solve_model()

k_mesh = bloch_wfs.k_mesh

u_wfs = bloch_wfs.get_states()["Cell periodic"]
u_wfs_occ = u_wfs[..., :n_occ, :]

In [3]:
U_links = bloch_wfs.get_links(state_idx=np.arange(n_occ))
print(U_links.shape)

Computing links for direction: mu=0
Applying phase
Computing links for direction: mu=1
Applying phase
(2, 10, 10, 4, 4)


In [77]:
abs(U_links[0, 7, 1]).round(3)

array([[0.998, 0.025, 0.055, 0.004],
       [0.023, 0.999, 0.045, 0.002],
       [0.056, 0.044, 0.997, 0.029],
       [0.003, 0.002, 0.029, 1.   ]])

In [76]:
abs(u_wfs_occ[7, 1].conj() @ u_wfs_occ[8, 1].swapaxes(-1,-2)).round(3)

array([[0.998, 0.025, 0.054, 0.004],
       [0.023, 0.997, 0.044, 0.002],
       [0.056, 0.044, 0.995, 0.03 ],
       [0.003, 0.002, 0.028, 0.98 ]])

In [ ]:
G = np.zeros(k_mesh.dim)
G[0] = 1
phase = np.exp(-2j * np.pi * G @ orb_vecs.T)
print(phase[np.newaxis, :].shape)

(1, 8)


In [72]:
abs(u_wfs_occ[-1, 1].conj() @ (u_wfs_occ[0, 1]* phase[np.newaxis, :]).swapaxes(-1,-2)).round(3)

array([[1.   , 0.002, 0.016, 0.001],
       [0.006, 0.9  , 0.419, 0.068],
       [0.015, 0.42 , 0.872, 0.227],
       [0.002, 0.039, 0.23 , 0.957]])

In [60]:
u_wfs_occ[0, 1].round(3)#* phase[np.newaxis, :]

array([[-0.417+0.j   ,  0.288-0.005j, -0.332-0.j   ,  0.287+0.002j,
        -0.435-0.j   ,  0.311-0.j   , -0.426-0.j   ,  0.289+0.003j],
       [ 0.666+0.j   , -0.195+0.j   ,  0.237-0.j   , -0.116-0.061j,
        -0.32 -0.j   ,  0.046+0.051j, -0.54 +0.j   ,  0.216+0.01j ],
       [ 0.075+0.j   , -0.19 +0.061j, -0.367-0.j   ,  0.318-0.041j,
         0.692+0.j   , -0.112+0.078j, -0.456-0.j   ,  0.053-0.098j],
       [-0.408+0.j   , -0.23 -0.088j,  0.663-0.j   ,  0.086+0.07j ,
         0.155+0.j   ,  0.32 +0.028j, -0.333-0.j   , -0.288-0.01j ]])

In [55]:
u_wfs_occ[0, 1].shape

(4, 8)

In [100]:
for mu in range(2):
    for nu in range(mu+1, 2):
        print(f"Computing flux in plane: mu={mu}, nu={nu}")
        U_mu = U_links[mu]
        U_nu = U_links[nu]

        print(U_mu.shape)

        U_nu_shift_mu = np.roll(U_nu, -1, axis=mu)
        U_mu_shift_nu = np.roll(U_mu, -1, axis=nu)

        U_wilson = np.matmul(
            np.matmul(
                np.matmul(U_mu, U_nu_shift_mu), U_mu_shift_nu.conj().swapaxes(-1, -2)
                ),
                U_nu.conj().swapaxes(-1, -2)
                )
        
        print(U_wilson.shape)
    
    
        # numpy
        eigvals, eigvecs = np.linalg.eig(U_wilson)
        print(eigvecs.shape)
        angles = -np.angle(eigvals)
        angles_diag = np.einsum("...i, ij -> ...ij", angles, np.eye(angles.shape[-1]))
        eigvecs_inv = np.linalg.inv(eigvecs)
        phases_rot = np.matmul(np.matmul(eigvecs, angles_diag), eigvecs_inv)
        print(angles[0,0].round(5))
        print(eigvecs[0,0].round(3))
        print(phases_rot[0,0].round(3))



Computing flux in plane: mu=0, nu=1
(10, 10, 4, 4)
(10, 10, 4, 4)
(10, 10, 4, 4)
[-0.       0.03961  0.02896  0.0232 ]
[[ 0.999+0.j     0.034-0.j    -0.017-0.003j  0.013-0.001j]
 [-0.001-0.001j -0.273-0.018j  0.087+0.022j  0.958+0.j   ]
 [ 0.001-0.001j  0.424-0.048j  0.904+0.j     0.037-0.001j]
 [-0.04 +0.j     0.861+0.j    -0.416-0.045j  0.285-0.022j]]
[[ 0.   +0.j -0.   -0.j  0.   -0.j  0.001-0.j]
 [-0.   +0.j  0.024+0.j -0.001-0.j -0.004-0.j]
 [ 0.   +0.j -0.001+0.j  0.031-0.j  0.004-0.j]
 [ 0.001+0.j -0.004+0.j  0.004+0.j  0.036-0.j]]


In [91]:
x = np.random.randn(4,4)

In [92]:
x

array([[-0.52464496,  0.84920905,  0.35173221,  0.04608124],
       [-0.5254752 , -1.28098972, -0.60928826, -0.521263  ],
       [ 0.26407351, -1.60796594, -0.18893616,  0.95185864],
       [ 1.04615091,  0.4221348 ,  0.17033133, -0.55780202]])

In [93]:
d = np.array([2, 3, 4, 5])

In [94]:
x * d[:, None]

array([[-1.04928993,  1.6984181 ,  0.70346442,  0.09216248],
       [-1.57642561, -3.84296916, -1.82786477, -1.56378901],
       [ 1.05629405, -6.43186377, -0.75574463,  3.80743457],
       [ 5.23075455,  2.11067402,  0.85165666, -2.78901011]])

In [95]:
x @ np.diag(d)

array([[-1.04928993,  2.54762715,  1.40692884,  0.2304062 ],
       [-1.05095041, -3.84296916, -2.43715303, -2.60631502],
       [ 0.52814702, -4.82389783, -0.75574463,  4.75929321],
       [ 2.09230182,  1.26640441,  0.68132532, -2.78901011]])

In [96]:
np.diag(d)

array([[2, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 0, 4, 0],
       [0, 0, 0, 5]])

In [97]:
np.einsum("...i, ij -> ...ij", d, np.eye(d.shape[-1]))

array([[2., 0., 0., 0.],
       [0., 3., 0., 0.],
       [0., 0., 4., 0.],
       [0., 0., 0., 5.]])